In [1]:
# -*- coding: utf-8 -*-

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import numpy as np
import time

def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)


/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
##################################################
BATCH_SIZE = 100
class_size = 4
learning_rate = 0.0003   # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
train_keep_prob = 0.7
max_len = 500           # sequence 단어 수 제한
max_vocab = 40000       # maximum 단어 개수
emb_dim = 128            # 단어 embedding dimension
hidden_dim = 128        # RNN hidden dim
train_acc_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_acc_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
train_loss_summary = [] # 학습용 데이터의 정확도를 중간 중간 기록한다
test_loss_summary = []  # 테스트용 데이터의 정확도를 중간 중간 기록한다
hypothesis_summary = []
##################################################
from ensemble_loader import movie_data
data = movie_data(max_vocab=max_vocab, max_len=max_len)

train_total_batch = int(data.train_size / BATCH_SIZE)
test_total_batch = int(data.test_size / BATCH_SIZE)

from genre_ensemble_model_v9 import Model
model = Model(max_len=max_len,
              emb_dim=emb_dim,
              hidden_dim=hidden_dim,
              vocab_size=max_vocab,
              class_size=class_size,
              use_clip=True, learning_rate=learning_rate)

sess = initialize_session()
sess.run(tf.global_variables_initializer())

Using TensorFlow backend.


x_train size 4800
y_train size 4800
x_test  size 1200
y_test  size 1200
x_ids_train  size 4800
x_len_train  size 4800
x_ids_test  size 1200
x_len_test  size 1200


In [3]:
def test_model():
    test_loss, test_acc = 0, 0

    for _ in range(test_total_batch):
        test_x_img, test_ids, test_length, test_y = data.get_test(BATCH_SIZE)

        loss, acc = sess.run([model.loss, model.accuracy],
                              feed_dict={model.x_image: test_x_img, model.x_ids: test_ids, model.x_len: test_length,
                              model.y_label: test_y, model.keep_prob : 1.0, model.is_training:False})
        test_loss += loss
        test_acc += acc

    return test_loss/test_total_batch, test_acc/test_total_batch
    #print(" * test loss: {:.3f} | test acc: {:.3f}\n".format(test_loss / test_cnt, sess.run(model.accuracy, feed_dict={model.x_image: test_x, model.y_label: test_label})))




In [4]:
for epoch in range(40):

    train_avg_loss, train_avg_acc = 0, 0

    for it in range(train_total_batch):
        train_x_img, train_ids, train_length, train_y = data.get_train(BATCH_SIZE)

        loss, acc, _, hypothesis_ = sess.run([model.loss, model.accuracy, model.update, model.genre_prob],
                                              feed_dict={model.x_image: train_x_img, model.x_ids: train_ids, model.x_len: train_length,
                                              model.y_label: train_y, model.keep_prob : train_keep_prob, model.is_training:True})
        train_avg_loss += loss / train_total_batch
        train_avg_acc += acc / train_total_batch

        #if it % 10 == 0 and it > 0:
            #model.save(sess)
            #print("*Model Saved train_loss: {:.4f}, train_acc: {:.4f}".format(loss/BATCH_SIZE, acc))

    test_avg_loss, test_avg_acc = test_model()

    train_acc_summary.append(train_avg_acc)
    test_acc_summary.append(test_avg_acc)
    train_loss_summary.append(train_avg_loss)
    test_loss_summary.append(test_avg_loss)
    hypothesis_summary.append(hypothesis_)

    print("epoch {} - train_loss: {:.4f}, train_acc: {:.4f}, test_loss: {:.4f}, test_acc: {:.4f}"
          .format(epoch+1, train_avg_loss, train_avg_acc, test_avg_loss, test_avg_acc))

epoch 1 - train_loss: 1.3145, train_acc: 0.3800, test_loss: 1.1980, test_acc: 0.4617
epoch 2 - train_loss: 1.0578, train_acc: 0.5631, test_loss: 0.9862, test_acc: 0.6167
epoch 3 - train_loss: 0.8422, train_acc: 0.7094, test_loss: 0.8869, test_acc: 0.6808
epoch 4 - train_loss: 0.6835, train_acc: 0.8081, test_loss: 0.8691, test_acc: 0.6925
epoch 5 - train_loss: 0.5558, train_acc: 0.8827, test_loss: 0.9016, test_acc: 0.6858
epoch 6 - train_loss: 0.4876, train_acc: 0.9281, test_loss: 0.9226, test_acc: 0.6758
epoch 7 - train_loss: 0.4697, train_acc: 0.9325, test_loss: 0.9059, test_acc: 0.6825
epoch 8 - train_loss: 0.4409, train_acc: 0.9554, test_loss: 0.9238, test_acc: 0.6992
epoch 9 - train_loss: 0.4134, train_acc: 0.9719, test_loss: 0.8806, test_acc: 0.7208
epoch 10 - train_loss: 0.4047, train_acc: 0.9796, test_loss: 0.9049, test_acc: 0.7142
epoch 11 - train_loss: 0.3974, train_acc: 0.9835, test_loss: 0.8704, test_acc: 0.7267
epoch 12 - train_loss: 0.3922, train_acc: 0.9856, test_loss: 0.